In [1]:
import numpy as np
import sys
from numpy.core.fromnumeric import shape, size
from random import random, randint
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


In [2]:
randint(-1000,1)

-752

In [74]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def Loss(Y, Y_predicted):
    return(sum(np.square(Y - Y_predicted))/Y.size).sum()

def rand(row, column):
    returned = np.empty((row, column))
    for j in range(row):
        rando=[]
        for i in range(column):
            rando.append(random())
            i+=1
        returned[j] = rando
    print("He",returned) 
    return returned


def Normalize(X):
    return((X-X.min())/(X.max()-X.min()))

def Binarize(X):
    X [X < 100] = 0
    X [X >= 100] = 1       
    return X

def calculate_accuracy(predicted_y, y):
    guessWrong = 0
    if(len(predicted_y) == len(y)):
        ind = 0
        for i in predicted_y:
            if(i != y[ind]):
                guessWrong = guessWrong + 1
            ind+=1
        acc = ( (len(predicted_y)- guessWrong) / len(predicted_y))
        return (acc*100)
    return 0

In [38]:
class Layer:
    nerweight =0
    inputs = 0
    outputs=0
    delta=0
    def __init__(self, neuronNumber,length):
        self.nerweight =rand(neuronNumber, length)
        #self.bais = rand(neuronNumber , 1)

    def feedForward(self, inputs):
        self.outputs= []
        self.inputs = inputs
        # print ("preFedd", self.outputs, inputs, self.nerweight)
        self.outputs  = sigmoid(np.dot(np.array(self.nerweight), self.inputs))
        # print ("Fedd", self.outputs)
        return self.outputs 

    
    def fit(self, eta, target, outj, sumDelta):
        if(size(sumDelta)==0):
            self.delta=(target-self.outputs)*self.outputs*(1-self.outputs)
            dw = eta*np.transpose([self.delta])*np.transpose(outj)
            self.nerweight = self.nerweight + dw 
            return np.dot(self.delta, self.nerweight)
        else:
            self.delta=sumDelta*self.outputs*(1-self.outputs)
            dw = eta*self.delta*np.transpose([outj])
            self.nerweight = self.nerweight + dw.reshape(len(self.nerweight), len(dw))
            return np.dot(self.delta, self.nerweight)
            

In [5]:
def getMax(ma):
    for j in range(shape(ma)[0]):
        cu = ma[j]
        ar=np.where(cu == np.amax(cu))
        for i in range (len(cu)):
            if(i == ar[0]):
                cu[i]=1
            else:
                cu[i]=0
        ma[j]= cu
    return ma

In [6]:
def accuracy_here(predicted_y, y):
    guess_wrong=0
    for i in range(len(predicted_y)):
        if(not np.array_equiv(predicted_y[i],y[i])):
            guess_wrong+=1
    acc = ((len(predicted_y)- guess_wrong) / len(predicted_y))
    return (acc*100)

In [86]:
class Network:
    layers = 0
    layernum=0
    length=0
    orin=0
    inputs =0
    target =0
    def __init__(self, NumNeuron, inputs, target):
        self.layers = []
        self.orin = inputs/100
        self.inputs =inputs
        self.target = target
        length = shape(inputs)[1]
        layer = Layer(4, length)  
        length =4
        self.layers.append(layer)
        for i in range (len(NumNeuron)):
            if(NumNeuron[i]<4):
                NumNeuron[i] = 4
            layer = Layer(NumNeuron[i], length)  
            length = NumNeuron[i]   
            self.layers.append(layer)
        layer = Layer(shape(target)[1], length)  
        self.layers.append(layer)

    def feedForward(self, datafeeded, data):
        current = data[datafeeded]
        for i in range(len(self.layers)):
            current= self.layers[i].feedForward(current)
        return current
    
    def fit(self, eta, leranRate):
        for j in range(leranRate):
            now = np.empty((shape(self.target)))
            for k in range((shape(self.orin)[0])):
                now[k] =self.feedForward(k, self.orin)
                sumdelta = []
                for i in range(len(self.layers)-1, 0, -1):
                    sumdelta =self.layers[i].fit(eta, self.target[k], self.layers[i-1].outputs, sumdelta)
                sumdelta =self.layers[0].fit(eta, self.target[k], self.orin[k], sumdelta)
            print (accuracy_here(getMax(now), train_y))


In [8]:
def centroid(X):
    M00=np.sum(X.sum(axis=1))
    M10 =np.array([])
    M01 =np.array([])
    for i in range(X.shape[0]):
        M10= np.append(M10, (X[i].sum(axis=0)*(i+1)))
        
    for i in range(X.shape[1]):
        M01 = np.append(M01,(X[:,i].sum(axis=0)*(i+1)))
    M10 = np.sum(M10)
    M01 = np.sum(M01)
    if(M00==0):
        Xc= 0
        Yc= 0
    else:
        Xc = M10/M00
        Yc = M01/M00
    return np.array([Xc, Yc])

In [9]:
def FeatureImage(X):
    vector=np.array([])
    for i in range(2):
        for j in range(4):              #0-7, 0--14  
            vector= np.append(vector, centroid(X[j*7:((j+1)*7) ,i*14:((i+1)*14)])) 
    return vector

In [10]:
def plot_digit(X, y, idx):
    plt.imshow(X[idx], cmap='Greys', interpolation='nearest')
    plt.title('true label: %d' % y[idx])
    plt.show()

In [11]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X.shape

(60000, 28, 28)

In [12]:
vec[0]

NameError: name 'vec' is not defined

In [ ]:
train_X =Binarize(train_X)
test_X = Binarize(test_X)

In [13]:
vec = np.array([])
for i in range(60000):
    vec = np.append(vec, FeatureImage(train_X[i]))
vec.shape

(960000,)

In [14]:
vec =vec.reshape(60000, 16)
vec.shape

(60000, 16)

In [15]:
vecTest = np.array([])
for i in range(10000):
    vecTest = np.append(vecTest, FeatureImage(test_X[i]))
print(vecTest.shape)
vecTest =vecTest.reshape(10000, 16)
print(vecTest.shape)

(160000,)
(10000, 16)


In [16]:
def fun (num):
    n = np.empty((len(num), 10))
    for i in range (len(num)):
        re = np.zeros((1 , 10))
        re[:,num[i]] = 1
        n[i] =re
    return n

In [17]:
test_y = fun(test_y)


In [18]:
vec[0]

array([ 6.97229551, 12.6055409 ,  2.92178018, 12.10367498,  6.23522727,
       13.36363636,  2.60348032,  9.98800473,  6.62568701,  5.63812554,
        2.53328895,  2.93608522,  4.24870541,  3.6051496 ,  1.14285714,
        1.60192616])

In [19]:
train_y =fun (train_y)

In [20]:
vec[0]

array([ 6.97229551, 12.6055409 ,  2.92178018, 12.10367498,  6.23522727,
       13.36363636,  2.60348032,  9.98800473,  6.62568701,  5.63812554,
        2.53328895,  2.93608522,  4.24870541,  3.6051496 ,  1.14285714,
        1.60192616])

In [55]:
mis3 = Network([2, 35,6,15,2], vec[0:1000], train_y[0:1000])

In [22]:
shape(train_y)

(60000, 10)

In [23]:
m = np.array( [ 7.  ,       13.,          3.,         12.12,        6.75,       13.25,
  2.69230769,  9.96153846,  6.5625 ,     5.6875,      3. ,         2.5,
  4.20689655,  3.62068966,  1.  ,        1.5       ])

mm = np.array([[[[4.77431391e-01, 1.04499075e-01, 3.88390791e-02, 4.86546162e-01,
  7.04942153e-01, 4.52568046e-01, 3.92915692e-01, 5.44536576e-01,
  8.81941246e-01, 4.60136774e-01, 5.34504601e-01, 8.65767106e-01,
  1.90331802e-01, 5.81820606e-01, 8.68753925e-01, 9.61046061e-01],
 [3.08960574e-01, 8.90150401e-01, 2.91677230e-02, 6.87794298e-01,
  3.08562704e-01, 3.32284070e-01, 5.77803042e-04, 6.94693826e-01,
  2.74574786e-01, 6.27544917e-02, 9.47873903e-01, 1.95163716e-01,
  4.70561663e-01, 7.84679191e-02, 4.33621294e-01, 1.71554074e-01]]]])

sigmoid(np.dot(np.array(mm), np.array(m/100)))

array([[[0.61145074, 0.60828428]]])

In [ ]:
4.77431391e-01*7

3.3420197369999998

In [70]:
mis3.fit(0.01, 50)


10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9
10.9


In [71]:
mis3.fit(0.02, 25)


10.5
11.0
10.7
11.200000000000001
11.200000000000001
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1
11.1


In [72]:
mis3.fit(3, 20)


9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001
9.700000000000001


11.200000000000001  --> 1000  --> 0.01 -->100

In [64]:
preY = np.empty((shape(test_y)))

In [65]:
for i in range(len(test_y)):
    preY[i]=mis3.feedForward(i, vecTest)


In [90]:
mis = Network([6], vec[0:1], train_y[0:1])

He [[0.28555906 0.56913656 0.56568328 0.54377316 0.7559556  0.33664438
  0.10340732 0.7648233  0.99068753 0.1100814  0.29965395 0.84589884
  0.05037755 0.30633048 0.39417435 0.04030266]
 [0.59616651 0.21160469 0.26377856 0.42344388 0.77739701 0.71557521
  0.92936855 0.23828593 0.39404311 0.77087659 0.63463955 0.38236958
  0.55931395 0.32358541 0.58745277 0.12541496]
 [0.99119782 0.47819255 0.16367875 0.75505107 0.40374841 0.79211952
  0.4193488  0.73080799 0.30409823 0.42767877 0.5327048  0.34045518
  0.90816001 0.46744367 0.94725444 0.64150044]
 [0.28882353 0.87966157 0.16177526 0.57551552 0.53195124 0.82849354
  0.95768472 0.32361299 0.04564711 0.60287946 0.23777434 0.48048853
  0.52751123 0.43010201 0.92173635 0.00901497]]
He [[0.92090732 0.73125516 0.6227915  0.69278729]
 [0.06160105 0.0150253  0.32332769 0.74627375]
 [0.32336095 0.27681919 0.54440973 0.48152213]
 [0.11142851 0.803625   0.56675866 0.61179549]
 [0.77345244 0.36331994 0.6406727  0.14358586]
 [0.34732161 0.97832626 0.

In [96]:
mis = Network([], vec[0:1000], train_y[0:1000])

He [[9.47394494e-01 7.35351586e-02 2.21061709e-01 4.06996763e-01
  4.18236040e-01 3.20925923e-01 2.22550541e-01 7.03589272e-01
  4.40841427e-01 8.03594517e-01 2.70882066e-01 6.24601903e-01
  2.53961886e-01 7.08960111e-01 5.36880358e-01 7.95370904e-01]
 [9.87755878e-01 9.31041923e-01 5.18618644e-01 7.90564408e-01
  3.28333389e-01 3.71945374e-01 7.84846087e-01 3.33609683e-04
  3.95496828e-01 2.54543886e-01 4.32408943e-01 5.20939865e-01
  8.35658640e-01 9.68934380e-01 9.51361673e-01 7.48630710e-01]
 [6.18435231e-01 6.44335593e-01 5.79202342e-01 4.15921743e-01
  4.95461293e-01 2.73077966e-01 8.63005901e-03 9.19615141e-01
  3.53105298e-01 2.76109938e-01 2.88469346e-01 1.48738941e-01
  8.86834825e-01 7.23472006e-01 4.55542809e-01 7.11212667e-01]
 [1.79121133e-01 4.80579006e-01 4.17160659e-01 4.78595112e-01
  6.07165595e-01 7.98370811e-01 8.98608606e-01 2.97295095e-01
  3.28223690e-01 3.26203672e-01 9.23950148e-01 8.65194019e-01
  2.78938053e-01 2.28977815e-01 3.04169191e-01 8.31696775e-01]]


In [97]:
mis.fit(0.01, 10)

9.2
9.2
9.2
11.0
11.1
10.7
10.7
10.9
10.9
11.0


In [89]:
len(mis.layers)

2

In [66]:
print (accuracy_here(getMax(preY), test_y))

10.280000000000001


In [ ]:
one = Network([2, 3,10], vec[0:1000], train_y[0:1000])

In [ ]:
one.fit(2, 10000)

31.1
31.900000000000002
32.4
32.5
32.5
32.6
32.6
32.800000000000004
32.9
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33

KeyboardInterrupt: 

In [ ]:
def accuracy_here(predicted_y, y):
    guess_wrong=0
    for i in range(len(predicted_y)):
        if(not np.array_equiv(predicted_y[i],y[i])):
            guess_wrong+=1
    acc = ((len(predicted_y)- guess_wrong) / len(predicted_y))
    return (acc*100)

11.0

In [ ]:
iris = load_iris() # Sepal Length, Sepal Width, Petal Length and Petal Width
X = iris.data
Y = iris.target
print(Y)
#My split function
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [ ]:
IRNet = Network([2,2], X_train, Y_train)